In [4]:
import pandas as pandas
import nltk
from functools import reduce

NOME_DO_ARQUIVO = 'noticias_estadao.csv'
indice_invertido = {}

A função que gera o índice usa a biblioteca Pandas para gerar um dataset com os dados do CSV. Através desse dataset, é feita uma leitura linha por linha. Fazendo uso da bibliteca NLTK, quebra-se os textos (titulo e descricao) em tokens, onde este token é adicionado como chave do indíce invertido e o valor deste índice é o id dos documentos que contém aquele termo.

In [5]:
def geraIndice():
    dados = pandas.read_csv(NOME_DO_ARQUIVO)
    for i in range(dados.titulo.size):
        texto = dados.titulo[i] + " " + dados.conteudo[i]
        palavras_divididas = nltk.word_tokenize(texto)
        for palavra in palavras_divididas:
            palavra = palavra.lower()
            if palavra not in indice_invertido:
                indice_invertido[palavra] = set([dados.idNoticia[i]])
            else:
                indice_invertido[palavra].add(dados.idNoticia[i])

A busca AND funciona através da leitura de uma lista de termos que deseja-se consultar. O resultado é uma lista de ids dos documentos que contém todos os termos buscados.

In [6]:
def busca_and(termos):
    resultado = reduce((lambda x,y: indice_invertido[x] & indice_invertido[y]), termos)
    return list(resultado)

Já a busca OR funciona retornando a união dos índices dos documentos que possuem os termos buscados, mas não necessariamente todos no mesmo texto.

In [7]:
def busca_or(termos):
    resultado = reduce((lambda x,y: indice_invertido[x].union(indice_invertido[y])), termos)
    return list(termos)

A maneira do usuário interagir e realizar buscar é digitando os termos que deseja buscar e, em seguida, escolhendo o tipo de pesquisa que deseja realizar. É impresso, então, a lista de documentos correspondentes à busca.

In [8]:
def read_input():
    print("Digite os termos que você deseja pesquisar, separados por espaço\n")
    termos = str(input()).split()
    print("Digite o número da operação desejada:\n1. AND\n2. OR")
    operador = int(input())
    if operador == 1:
        print(busca_and(termos))
    else:
        print(busca_or(termos))
    
def testes():
    assert len(busca_or(["debate", "presidencial"])) == 1770
    assert len(busca_and(["debate", "presidencial"])) == 201
    assert len(busca_or(["presidenciáveis", "corruptos"])) == 164
    assert len(busca_and(["presidenciáveis", "corruptos"])) == 0
    assert len(busca_and(["belo", "horizonte"])) == 242
    assert len(busca_or(["belo", "horizonte"])) == 331
    
if __name__ == "__main__":
    geraIndice()
    testes()
    read_input()

AssertionError: 

In [16]:
(busca_or(["belo", "horizonte"]))

['belo', 'horizonte']